In [1]:
import pandas as pd
import numpy as np
import pickle
import openai
import os
import re

from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
with open("02450_data.pkl", "rb") as fp:
    data_02450 = pickle.load(fp)

In [3]:
n_questions = data_02450.shape[0]
n_contexts = data_02450[data_02450['Context'].values!=None].shape[0]

In [4]:
df_context = pd.DataFrame(data={'context':data_02450['Context'],'embedding':data_02450['Context embeddings']})
df_context.drop(0, inplace=True)
df_context.dropna(inplace=True)

In [100]:
df_question = pd.DataFrame(data={'question':data_02450['Question'],'embedding':data_02450['Question embeddings']})
test_questions = df_question.sample(n=5)
gen_questions = df_question.sample(n=5)
df_question.drop(test_questions.index, inplace=True)
df_question.drop(gen_questions.index, inplace=True)

In [104]:
data = []
for q,qe in df_question.values:
    for c,ce in df_context.values:
        data.append([q,c,qe,ce,None])
df = pd.DataFrame(data=data, columns=['question','context','question_embedding','context_embedding','score'])

In [105]:
data = []
for q,qe in test_questions.values:
    for c,ce in df_context.values:
        data.append([q,c,qe,ce,None])
df_test = pd.DataFrame(data=data, columns=['question','context','question_embedding','context_embedding','score'])

In [106]:
data = []
for q,qe in gen_questions.values:
    for c,ce in df_context.values:
        data.append([q,c,qe,ce,None])
df_gen = pd.DataFrame(data=data, columns=['question','context','question_embedding','context_embedding','score'])

In [8]:
print('Mean context length (chars):',df['context'].apply(lambda x: len(x)).values.mean())

Mean context length (chars): 4746.046357615894


In [13]:
fred_ct = 'A Bayesian network also called a belief network or Bayesian belief network is not as such adding a new method to our toolbox, but it provides a convenient and often-used notation for presenting existing probabilities. Consider the following example adapted from Pearl [2014], MacKay [2003] Fred lives in Los Angeles and commutes 60 miles to work. Whilst at work, he receives a phone-call from his neighbour saying that Fred’s burglar alarm is ringing. What is the probability that there was a burglar in his house today? While driving home to investigate, Fred hears on the radio that there was a small earthquake that day near his home. ‘Oh’, he says, feeling relieved, ‘it was probably the earthquake that set off the alarm’. What is the probability that there was a burglar in his house? To analyse this story we first introduce the variables: a : The alarm is ringing. b : A burglar was in Fred’s house. c : Fred received a phone-call reporting the alarm. e : A small earthquake took place today near Fred’s house. r : The radio report of the earthquake is heard by Fred. In a case like this, we know (from our experience) that some of these events must be independent. That there is a burglar or a minor earthquake is presumable unrelated events, so p(b, e) = p(b)p(e). In general, the probability of these variables will factorize as follows:232 13 Bayesian methods Earthquake Burglar Radio Alarm Phonecall Fig. 13.3. Bayesian network of the burglar example. Each vertex corresponds to a variable, and incident edges corresponds to conditional dependence. p(a, b, c, e, r) = p(b)p(e)p(a|b, e)p(c|a)p(r|e). (13.14) This factorization of the probability has important consequences. Firstly, as for the na¨ıve-Bayes assumption, it makes the probability density much less costly to store on a computer and reliable to estimate as there are fewer parameters than the full (un-factorized) joint distribution. Secondly, it allows faster computation by exploiting the factorization structure and finally it allows us easier to see what quantities are independent of each other. It is common to represent the factorization as a network where the vertices correspond to the variables and the edges correspond to statistical dependence, see fig. 13.3 for an illustration. So for instance, if there is an edge from B to A, that means that in the joint distribution, then A must be conditional on B and possible other variables connected to A. To solve the Burglar problem, assume the probability of there being a burglar is p(b = 1) = 0.1%, earthquake p(e = 1) = 0.1% (corresponding to roughly one burglar and earthquake every four years) and that the alarm is triggered either by (1) false alarms (very low probability), (2) if an earthquake takes place (low probability) and finally if a burglar enters the home (high probability). In our example these probabilities are:1 1 For instance, suppose we let f = 0.1% denote the chance a false alarm triggers a, αe = 1% the chance an earthquake triggers a and finally αb = 99% the chance a burglar triggers a. The probabilities can then be obtained as: p(a = 1|b = 0, e = 0) = f, p(a = 1|b = 0, e = 1) = 1−(1−f)(1−αe), p(a = 1|b = 1, e = 0) = 1−(1−f)(1−αb), p(a = 1|b = 1, e = 1) = 1−(1−f)(1−αb)(1−αe).13.3 Bayesian networksF 233 p(a = 1|b = 0, e = 0) = 0.1%, p(a = 1|b = 0, e = 1) = 1.099%, p(a = 1|b = 1, e = 0) = 99.001%, p(a = 1|b = 1, e = 1) = 99.011%. Finally assume the neighbour would never phone if the alarm is not ringing (p(c = 1|a = 0) = 0) and that the radio reported is also trustworthy (p(r = 1|e = 0) = 0) and let’s return to the problem: Suppose first the phone calls c = 1; then we know the alarm is ringing a = 1 and so the posterior probability of b, e (burglary and earthquake) becomes: p(b, e|a = 1) = p(a = 1|b, e)p(b, e) p(a = 1) . We can use the Bayes network to compute these probabilities. For instance when computing p(a = 1), we must compute this by summing over all other variables than a: p(a = 1) = X b∈{0,1} X c∈{0,1} X e∈{0,1} X r∈{0,1} p(a = 1, b, c, e, r), (13.15) However, if we plug in the expression of the likelihood (13.14) we see that variables c and r can trivially be summed out (i.e., marginalized): p(a = 1) = X b∈{0,1} X c∈{0,1} X e∈{0,1} X r∈{0,1} p(b)p(e)p(a = 1|b, e)p(c|a = 1)p(r|e) = X b∈{0,1} X e∈{0,1} \uf8ee \uf8f0p(b)p(e)p(a = 1|b, e) \uf8eb \uf8ed X c∈{0,1} p(c|a = 1) X r∈{0,1} p(r|e) \uf8f6 \uf8f8 \uf8f9 \uf8fb = X b∈{0,1} X e∈{0,1} p(b)p(e)p(a = 1|b, e) (13.16) Comparing to the Bayesian network in fig. 13.3 we see that to determine what variables remain in the sum when computing p(a), we look at all other vertices in a network where we can move to a by going in the direction of the edges. See fig. 13.4 where we have illustrated the two nodes that remain, e, b, with red. Using the above numbers we obtain: p(a = 1|b = 0, e = 0)p(b = 0)p(e = 0) = 0.000998, p(a = 1|b = 1, e = 0)p(b = 1)p(e = 0) = 0.0000989, p(a = 1|b = 0, e = 1)p(b = 0)p(e = 1) = 0.000010979, p(a = 1|b = 1, e = 1)p(b = 1)p(e = 1) = 9.9 × 10−7 . By inserting these four numbers into eq. (13.16) and summing we obtain p(a = 1) = 0.002 and so from eq. (13.15) p(b = 0, e = 0|a = 1) = 0.4993, (13.17) p(b = 1, e = 0|a = 1) = 0.4947, (13.18) p(b = 0, e = 1|a = 1) = 0.0055, (13.19) p(b = 1, e = 1|a = 1) = 0.0005. (13.20)234 13 Bayesian methods Earthquake Burglar Radio Alarm Phonecall Fig. 13.4. To determine what variables must be summed out when computing the marginal of a variable such as a, we look at all variables such that one can move in the direction of the arrows from those variables to a. This gives p(a, e, b) So returning to the initial question, when we determine if there was a burglar at the house we must compute p(b = 1|a = 1) which can be accomplished by marginalizing over the burglar-variable: p(b = 0|a = 1) = p(b = 0, e = 0|a = 1) + p(b = 0, e = 1|a = 1) = 0.505 p(b = 1|a = 1) = p(b = 1, e = 0|a = 1) + p(b = 1, e = 1|a = 1) = 0.494 so after receiving the call, we believe there to be a 50% chance there was a burglar in the house. An important point to take away from this example is that b and e, which were initially independent: p(e, b) = p(e)p(b), are made dependent by the information a. Now consider the final part of the example. Suppose we also learn that e = 1 (i.e. there was an earthquake). The probability there was a burglar can now be computed as: p(b|e, a) = p(b, e|a) p(e|a) = p(b, e|a) p(e, b = 0|a) + p(e, b = 1|a) . If we plug in numbers we obtain p(e = 1|a = 1) = 0.006 and so p(b = 0|e = 1, a = 1) = p(b = 0, e = 1|a = 1) p(e = 1|a = 1) = 0.92 p(b = 1|e = 1, a = 1) = p(b = 1, e = 1|a = 1) p(e = 1|a = 1) = 0.08 So after learning the alarm was triggered, this lowers our probability there was a burglar in the house from about 50% to about 8%. This is in according to everyday intuition: when we learn about the earthquake, we consider that to be the more plausible explanation of the alarm.'

# Oracle - Getting score from ChatGPT

In [ ]:
for i in range(0,len(df_test),2):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {"role":"system", "content":"You are a system designed to give a score in percentage that expresses how likely a list of provided question can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `score: PERCENTAGE"},
        {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a blue car?```"},
        {"role":"assistant", "content":"score: 95%\nscore: 60%"},
        {"role": "user", "content": f'CONTEXT: ```{df_test.iloc[i]["context"]}``` QUESTION: ```{df_test.iloc[i]["question"]}``` QUESTION: ```{df_test.iloc[i+1]["question"]}```'}
    ]
    )

    for li, l in enumerate(completion.choices[0].message.content.splitlines()):
        m = re.findall(r'score: (\d{1,2,3})%', l)
        if m:
            df_test.iloc[i+li]['score'] = m[0]
    
    if i % 10 == 0:
        print(i)

In [ ]:
for i in range(0,len(df_gen),2):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {"role":"system", "content":"You are a system designed to give a score in percentage that expresses how likely a list of provided question can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `score: PERCENTAGE"},
        {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a blue car?```"},
        {"role":"assistant", "content":"score: 95%\nscore: 60%"},
        {"role": "user", "content": f'CONTEXT: ```{df_gen.iloc[i]["context"]}``` QUESTION: ```{df_gen.iloc[i]["question"]}``` QUESTION: ```{df_gen.iloc[i+1]["question"]}```'}
    ]
    )

    for li, l in enumerate(completion.choices[0].message.content.splitlines()):
        m = re.findall(r'score: (\d{1,2,3})%', l)
        if m:
            df_gen.iloc[i+li]['score'] = m[0]
    
    if i % 10 == 0:
        print(i)

In [31]:
idx = list(df.index)
np.random.shuffle(idx)

In [37]:
for i in range(0,500,2):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {"role":"system", "content":"You are a system designed to give a score in percentage that expresses how likely a list of provided question can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `score: PERCENTAGE"},
        {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a blue car?```"},
        {"role":"assistant", "content":"score: 95%\nscore: 60%"},
        {"role": "user", "content": f'CONTEXT: ```{df.iloc[idx[i]]["context"]}``` QUESTION: ```{df.iloc[idx[i]]["question"]}``` QUESTION: ```{df.iloc[idx[i+1]]["question"]}```'}
    ]
    )

    for li, l in enumerate(completion.choices[0].message.content.splitlines()):
        m = re.findall(r'score: (\d{1,2,3})%', l)
        if m:
            df.iloc[idx[i+li]]['score'] = m[0]
    
    if i % 10 == 0:
        print(i)

score: 95%
score: 0%


In [21]:
# completion = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   temperature=0,
#   messages=[
#     {"role":"system", "content":"You are a system designed to give a score in percentage that expresses how likely a provided question can be answered using ANY part of a provided context. You will always only reply in the following format: `score: PERCENTAGE"},
#     {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?```"},
#     {"role":"assistant", "content":"label: 95%"},
#     {"role": "user", "content": f'CONTEXT: ```{ct}``` QUESTION: ```What is Bayesian Networks?```'}
#   ]
# )

# print(completion.choices[0].message.content)


score: 100%
